In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/m2cw

Mounted at /content/drive
/content/drive/MyDrive/m2cw


In [2]:
import h5py
import numpy as np
import torch
from transformers import AutoTokenizer
import wandb
import matplotlib.pyplot as plt
from src.preprocessor import load_and_preprocess
from src.lora_skeleton import apply_lora, train_lora,load_data,LoRALinear
from src.qwen import load_qwen
from src.evaluation import evaluation

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

wandb.login()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: liuyihao649 (liuyihao649-university-of-cambridge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
np.random.seed(42)

# Load Qwen2.5 0.5B Model and Tokenizer ---
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

file_path = "lotka_volterra_data.h5"

# Use the function to load and preprocess the data
train_texts, val_texts, test_texts = load_and_preprocess(
    file_path,
    decimal_places=2,
    max_target_value=9.99
)

# Demonstrate tokenization using Qwen2.5
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenized_train=[]
tokenized_val=[]
tokenized_test=[]
for i in range(len(train_texts)):
    tokenized_train.append(tokenizer(train_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])
for i in range(len(val_texts)):
    tokenized_val.append(tokenizer(val_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])
for i in range(len(test_texts)):
    tokenized_test.append(tokenizer(test_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])



tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
train_ids, val_ids, test_ids = load_data(tokenizer)

In [5]:
# Save the weights for each model
# Only saving LoRA part
def save_lora_parameters(model, file_path):
    """
    Saves only the LoRA parameters from the model.

    This function iterates over all modules in the model and extracts the parameters
    from those modules that are instances of LoRALinear. It saves the 'A' and 'B' parameters
    into a state dictionary and then saves it using torch.save.

    Args:
        model: The model containing LoRA modules.
        file_path (str): The file path to save the LoRA parameters.
    """
    state_dict = {}
    for name, module in model.named_modules():
        # Check if the module is an instance of LoRALinear
        if isinstance(module, LoRALinear):
            state_dict[name + ".A"] = module.A.detach().cpu()
            state_dict[name + ".B"] = module.B.detach().cpu()
    # Also save LM head's trainable parameters
    if hasattr(model, "lm_head") and hasattr(model.lm_head, "bias"):
        if model.lm_head.bias.requires_grad:
            state_dict["lm_head.bias"] = model.lm_head.bias.detach().cpu()
    torch.save(state_dict, file_path)


### 3(b): Grid search

In [ ]:
for rank in [2, 4, 8]:
    for lr in [1e-5, 5e-5, 1e-4]:
        # init each experiments
        wandb.init(
            project="Qwen-lora-grid-search",
            name=f"rank={rank}_lr={lr}_train",
            group="train_loss_curve",
            reinit=True
        )

        # load the original model and apply lora to it
        base_model,_ = load_qwen()
        apply_lora(base_model, r=rank)



        # LoRA train
        _ = train_lora(
            base_model,
            train_ids,
            learning_rate=lr,
            batch_size=4,
            max_steps=1000,

        )

        # Save the weights(only LoRA part, no need to save the whole model)
        save_path = f"/content/drive/MyDrive/m2cw/lora_rank={rank}_lr={lr}.pt"
        save_lora_parameters(base_model, save_path)

        # Evaluation
        wandb.init(project="Qwen-lora-grid-search",
            name=f"rank={rank}_lr={lr}_evaluation",
            reinit=True)
        val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


        print(f"rank{rank},lr{lr},evalaution results:")
        print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
        print(f"Average MSE (Forecast):  {val_mse:.4f}")



        wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Training:   5%|▌         | 50/1000 [00:15<04:37,  3.42it/s, loss=0.71]

Step 50: loss = 0.7103


Training:  10%|█         | 100/1000 [00:30<04:23,  3.42it/s, loss=0.683]

Step 100: loss = 0.6827


...

Training: 100%|██████████| 1000/1000 [04:52<00:00,  3.42it/s, loss=0.323]


Step 1000: loss = 0.3227


loss,█▇▄▅▅▄▅▅▄▄▂▄▃▃▃▃▁▃▃▂▃▁▂▂▂▃▂▂▂▂▃▂▂▂▃▁▂▁▂▂
step,▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇███
loss,0.32271
step,999


rank8,lr0.0001,evalaution results:
Average Cross-Entropy Loss: 1.6746
Average MSE (Forecast):  0.0071


avg_loss,▁
avg_mse,▁
avg_loss,1.67464
avg_mse,0.00707


# Now we select the best hyperparameter settings to train models again by varying the length of sequence of training set.

In [ ]:
max_context_length=[128,512,768]

for length in max_context_length:
    train_ids,_,_=load_data(tokenizer,max_ctx_length=length)
    wandb.init(
        project="Qwen-lora-different-context-len",
        name=f"context-length={length}_train",
        group="train_loss_curve",
        reinit=True
    )
    base_model,_ = load_qwen()
    apply_lora(base_model,r=8)
    _ = train_lora(
        base_model,
        train_ids,
        learning_rate=1e-4,
        batch_size=4,
        max_steps=1000,
    )
    # Save the weights(only LoRA part, no need to save the whole model)
    save_path = f"/content/drive/MyDrive/m2cw/Context_length={length}_rank=2_lr=1e-4.pt"
    save_lora_parameters(base_model, save_path)


    wandb.init(
        project="Qwen-lora-different-context-len",
        name=f"context-length:{length}_evaluation",
        reinit=True
    )
    val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


    print(f"rank{rank},lr{lr},evalaution results:")
    print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
    print(f"Average MSE (Forecast):  {val_mse:.4f}")

    wandb.finish()


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


Training:   5%|▌         | 51/1000 [00:05<01:44,  9.05it/s, loss=0.598]

Step 50: loss = 0.6213


Training:  10%|█         | 101/1000 [00:11<01:40,  8.93it/s, loss=0.555]

Step 100: loss = 0.5399


...

Training: 100%|██████████| 1000/1000 [07:01<00:00,  2.37it/s, loss=0.274]


Step 1000: loss = 0.2743


loss,█▆▄▅▄▅▃▃▃▃▁▁▃▃▂▂▂▅▄▃▄▃▁▃▃▂▃▃▂▂▄▂▃▂▄▂▁▂▂▂
step,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇███
loss,0.27428
step,999


rank8,lr0.0001,evalaution results:
Average Cross-Entropy Loss: 1.5362
Average MSE (Forecast):  0.0051


avg_loss,▁
avg_mse,▁
avg_loss,1.53623
avg_mse,0.00513


# 3(c) By using the context length and hyperparameter, we train the model again with 3000 steps.

In [6]:
wandb.init(
    project="Qwen-lora-final-long-run-8-1e-4-768",
        name=f"final_train",
        reinit=True)
train_ids,_,_=load_data(tokenizer,max_ctx_length=768)

base_model,_ = load_qwen()

apply_lora(base_model,r=8)

_ = train_lora(
    base_model,
    train_ids,
    learning_rate=1e-4,
    batch_size=4,
    max_steps=3000
)

# Save the weights(only LoRA part, no need to save the whole model)
save_path = f"/content/drive/MyDrive/m2cw/Final_context=768_rank=8_lr=1e-4.pt"
save_lora_parameters(base_model, save_path)


wandb.init(
    project="Qwen-lora-final-long-run-8-1e-4-768",
    name=f"final_evaluation_validation_set",
    reinit=True
)
val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)

wandb.init(
    project="Qwen-lora-final-long-run-8-1e-4-768",
    name=f"final_evaluation_test_set",
    reinit=True
)

test_loss, test_mse = evaluation(base_model, tokenizer, tokenized_test)

print(f"final evalaution (val) results:")
print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
print(f"Average MSE (Forecast):  {val_mse:.4f}")


print(f"final evalaution (test) results:")
print(f"Average Cross-Entropy Loss: {test_loss:.4f}")
print(f"Average MSE (Forecast):  {test_mse:.4f}")

wandb.finish()


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Training:   2%|▏         | 50/3000 [00:22<20:38,  2.38it/s, loss=0.485]

Step 50: loss = 0.4854


Training:   3%|▎         | 100/3000 [00:43<20:17,  2.38it/s, loss=0.699]

Step 100: loss = 0.6991


...

loss,█▅▇▅▆▄▆▅▅▄▅▆▄▅▆▃▆▁▃▄▂▃▄▄▅▃▄▃▄▅▁▃▃▄▅▃▃▂▃▂
step,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇██
loss,0.25688
step,2999


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos toke

avg_loss,▁
avg_mse,▁
avg_loss,1.21281
avg_mse,0.00209


final evalaution (val) results:
Average Cross-Entropy Loss: 1.2128
Average MSE (Forecast):  0.0021
final evalaution (test) results:
Average Cross-Entropy Loss: 1.9080
Average MSE (Forecast):  0.0071


avg_loss,▁
avg_mse,▁
avg_loss,1.90797
avg_mse,0.00715
